# Practice -- working with databases in Python

In [13]:
import sqlite3
import pandas as pd

In [14]:
con = sqlite3.connect('imdb_small_indexed.db')  # connecting
cur = con.cursor()  # cursor

In [20]:
tom_hanks_query = """
SELECT title, premiered
FROM titles 
    JOIN crew ON titles.title_id = crew.title_id
    JOIN people ON crew.person_id = people.person_id
WHERE name = "Tom Hanks"
ORDER BY premiered DESC
LIMIT 5
"""

**option 1**: get everything at once

In [19]:
cur.fetchall()

[('BIOS', 2021),
 ('Untitled Elvis Presley Project', 2021),
 ('Borat Subsequent Moviefilm', 2020),
 ('Greyhound', 2020)]

A list of tuples is returned.

**option 2**: only one 

In [18]:
cur.execute(tom_hanks_query)
cur.fetchone()

('Untitled Elvis Presley Project', 2021)

Iterating using fetchone



In [23]:
while True:
    result = cur.fetchone()
    if result is not None:
        print(result)
    else:
        break

**option 3**: a portion

In [22]:
cur.execute(tom_hanks_query)
while True:
    result = cur.fetchmany(2)
    if len(result) > 0:
        print(result)
    else:
        break

[('Untitled Elvis Presley Project', 2021), ('BIOS', 2021)]
[('Untitled Elvis Presley Project', 2021), ('Borat Subsequent Moviefilm', 2020)]
[('Greyhound', 2020)]


### Problem 1

Select all the films after 2000, where your favourite actor or actrice is playing. Print all the results.

In [ ]:
# Your code here

## Using variables 



In [24]:
name = "Natalie Portman"
genre = "Comedy"

In [25]:
act_genre_query = """
SELECT title, premiered
FROM titles 
    JOIN crew ON titles.title_id = crew.title_id
    JOIN people ON crew.person_id = people.person_id
    JOIN film_genres ON titles.title_id = film_genres.title_id
    JOIN genre_types ON film_genres.genre_id = genre_types.id
WHERE name = ? AND genre_name = ? AND premiered IS NOT NULL
ORDER BY premiered DESC
"""

In [26]:
cur.execute(act_genre_query, (name, genre))
cur.fetchall()

[('Your Highness', 2011),
 ('No Strings Attached', 2011),
 ('Hesher', 2010),
 ('New York, I Love You', 2008),
 ('Between Two Ferns with Zach Galifianakis', 2008),
 ("Mr. Magorium's Wonder Emporium", 2007),
 ('Free Zone', 2005),
 ('Garden State', 2004),
 ('Where the Heart Is', 2000),
 ('Anywhere But Here', 1999)]

### Problem 2

Write a function that takes a type of film, genre, rating and returns the films that were rated by more than 100 000 people and with a rating higher than your specified number.

In [ ]:
# Your code goes here

### SQL and pandas



In [27]:
df = pd.read_sql_query(act_genre_query, params = [name, genre], con=con)
df

,title,premiered
0,Your Highness,2011
1,No Strings Attached,2011
2,Hesher,2010
3,"New York, I Love You",2008
4,Between Two Ferns with Zach Galifianakis,2008
5,Mr. Magorium's Wonder Emporium,2007
6,Free Zone,2005
7,Garden State,2004
8,Where the Heart Is,2000
9,Anywhere But Here,1999


### Problem 3

Rework problem 2 so that the data is in a dataframe format: title, rating, premiered

In [ ]:
# Your code goes here

## Creating your own tables

Use ```CREATE```

The template:

``` sql
CREATE TABLE table_name (column1 INT, column2 TEXT)
```

In [28]:
cur.execute("""
CREATE TABLE new_comedies (
    title_id INT, 
    title TEXT, 
    premiered INT,
    rating REAL,
    PRIMARY KEY (title_id)
)
""")

To save:

In [ ]:
con.commit()

Bafore you do the commit, you can rollback to the previously saved version:

In [ ]:
con.rollback()

In [29]:
new_comedies_query = """
SELECT titles.title_id, title, premiered, rating
FROM titles
    JOIN film_genres ON film_genres.title_id = titles.title_id
    JOIN genre_types ON film_genres.genre_id = genre_types.id
    JOIN rating ON titles.title_id = rating.title_id
WHERE genre_types.genre_name = "Comedy" AND titles.premiered > 2019 AND titles.premiered < 2021
"""

In [30]:
cur.execute(new_comedies_query)
data = cur.fetchall()

data[:5]

[(446792, 'Surviving in L.A.', 2020, 8.1),
 (805647, 'The Witches', 2020, 5.2),
 (10004368, 'Jak najdalej stad', 2020, 7.2),
 (10006418, 'Avocado Toast the series', 2020, 7.3),
 (10023150, 'My Cousin', 2020, 5.8)]

To add something into a table, use ```INSERT```:

In [31]:
cur.executemany("INSERT INTO new_comedies VALUES (?, ?, ?, ?)", data)
con.commit()

## Deleting data

Let's say that we want to delete all the films that start with a B.

In [32]:
cur.execute("SELECT COUNT(title_id) FROM new_comedies WHERE title LIKE 'B%'")
cur.fetchall()

[(73,)]

In [33]:
cur.execute("DELETE FROM new_comedies WHERE title LIKE 'B%'")
con.commit()

In [34]:
cur.execute("SELECT COUNT(title_id) FROM new_comedies WHERE title LIKE 'B%'")
cur.fetchall()

[(0,)]

To delete the whole table, use ```DROP```. To avoid an error if such a table does not exist, use ```IF EXISTS```:

In [35]:
cur.execute("DROP TABLE IF EXISTS new_comedies")
con.commit()

## Other functions

+ ```GROUP_CONCAT``` -- joins the data

+ ```HAVING``` -- for filtering the data


**Top-10 films based on their rating, where there are exactly two genres and 100 000 votes**


In [36]:
cur.execute("""
SELECT title, premiered, COUNT(film_genres.genre_id) as n_genres, rating, votes, GROUP_CONCAT(genre_name, ', ')
FROM titles
    JOIN film_genres ON titles.title_id = film_genres.title_id
    JOIN genre_types ON film_genres.genre_id = genre_types.id
    JOIN rating ON titles.title_id = rating.title_id
WHERE rating.votes > 100000
GROUP BY titles.title_id
HAVING n_genres = 2
ORDER BY rating DESC, votes DESC
LIMIT 10
""")
cur.fetchall()

[('The Godfather', 1972, 2, 9.2, 1585284, 'Crime, Drama'),
 ('The Sopranos', 1999, 2, 9.2, 290309, 'Crime, Drama'),
 ('The Godfather: Part II', 1974, 2, 9.0, 1107605, 'Crime, Drama'),
 ('Pulp Fiction', 1994, 2, 8.9, 1793495, 'Crime, Drama'),
 ('Friends', 1994, 2, 8.9, 794054, 'Comedy, Romance'),
 ('12 Angry Men', 1957, 2, 8.9, 674893, 'Crime, Drama'),
 ('Forrest Gump', 1994, 2, 8.8, 1769908, 'Drama, Romance'),
 ('Peaky Blinders', 2013, 2, 8.8, 316333, 'Crime, Drama'),
 ('Freaks and Geeks', 1999, 2, 8.8, 126761, 'Comedy, Drama'),
 ('The Matrix', 1999, 2, 8.7, 1644392, 'Action, Sci-Fi')]

### Problem 4

Select a couple of actors, create a table with year, rating, genres and your selected actors. You can edit the code below:

``` sql
SELECT 
    ...
    COUNT(...) as n_my_actors,
    GROUP_CONCAT(...) as my_actors
FROM titles 
    ...
WHERE name IN (...)
GROUP BY titles.title_id
HAVING n_my_actors > 1
ORDER BY n_my_actors, premiered DESC
```

In [ ]:
# Your code goes here